In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

## Dummy Data Creation

In [2]:
df = pd.read_csv('combined_sales_2017-2022.csv')

C:\Users\giova\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df[df['No. Rangka'] == 'MHKE8FB3JKK023250'].loc[:,['Cabang', 'Tgl DO', 'Model']]

,Cabang,Tgl DO,Model
12519,DPK,2019-02-28,Rush


In [4]:
date_cols = ['Tgl Input Spk', 'Tgl Create Customer', 'Tgl Lahir', 'Tgl DO', 'Tgl Spk',]

In [5]:
for col in date_cols:
    df[col] = pd.to_datetime(df[col])

In [6]:
df['Cabang'] = df['Cabang'].replace(['DPK', 'BGR', 'CBR', 'CMG', 'PRG'], 
                                    ['Jakarta', 'Bandung', 'Palembang', 'Semarang', 'Malang'])

In [7]:
df = df[(df['Model'] == 'Calya') | (df['Model'] == 'Rush') | 
   (df['Model'] == 'Yaris') | (df['Model'] == 'Avanza') |
   (df['Model'] == 'Sienta') | (df['Model'] == 'Fortuner') |
   (df['Model'] == 'Agya') | (df['Model'] == 'Kijang Innova')]

In [8]:
df['Model'] = df['Model'].replace(['Calya', 'Rush', 'Yaris', 'Avanza', 'Sienta', 'Fortuner', 'Agya',
                                     'Kijang Innova'], 
                                     ['Outlander PHEV', 'Outlander Sport', 'L300', 'Xpander', 'Triton 4x4', 'Pajero Sport', 'Eclipse Cross',
                                     'Xpander Cross'])

In [9]:
np.random.seed(10)

remove_n = 3000
drop_indices = np.random.choice(df.index, remove_n, replace=False)
df = df.drop(drop_indices)

In [10]:
shuf_cols = ['Cabang', 'Tgl DO', 'Model']

In [11]:
def random_dates(start, end, n=10):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

In [12]:
start = df['Tgl DO'].min()
end = df['Tgl DO'].max()
rand_date = random_dates(start, end, len(df))

In [13]:
for col in shuf_cols:
    df[col] = np.random.permutation(df[col])

In [14]:
df['Tgl DO'] = pd.to_datetime(rand_date).date

In [15]:
df = df.rename({'Cabang' : 'Showroom'}, axis = 1)

In [16]:
df.to_csv('dummy_sales_data.csv')

## Dummy Data Decomposition

In [17]:
showrooms = df['Showroom'].unique().tolist()

In [18]:
df_agg = df.groupby(["Showroom", "Tgl DO"]).count()['No. Rangka'].reset_index()
df_agg['Tgl DO'] = df_agg['Tgl DO'].astype('datetime64')

In [19]:
for showroom in showrooms:
    exec("df_{} = df_agg[df_agg['Showroom'] == '{}'].reset_index(drop = True).\
    iloc[:,1:].set_index('Tgl DO')".format(showroom, showroom))
    exec("df_{}.columns = ['{}']".format(showroom, showroom))

In [20]:
for showroom in showrooms:
    exec("df_{} = df_{}.reindex(pd.date_range(df_Jakarta.index.min(), df_{}.index.max()), fill_value = 0)"\
         .format(showroom,showroom, showroom, showroom))

In [21]:
idx_sd = pd.date_range(df_agg['Tgl DO'].min(), df_agg['Tgl DO'].max(), freq = 'D')

In [22]:
df_sd = pd.DataFrame()
for showroom in showrooms:
    exec("sd_30 = seasonal_decompose(x = df_{}, period = 30, model = 'additive', extrapolate_trend= 'freq')".format(showroom))
    exec("sd_365 = seasonal_decompose(x = df_{}, period = 365, model = 'additive')".format(showroom))
    sd_dict = {'Tgl DO' : idx_sd, \
                   'Showroom': showroom, 
                   'Observation': sd_30.observed,  
                   'Trend' : sd_30.trend, 
                   'Monthly Seasonal': sd_30.seasonal, 
                   'Yearly Seasonal' : sd_365.seasonal}

    df_temp = pd.DataFrame(sd_dict)
    df_sd = pd.concat([df_sd, df_temp])
df_sd = df_sd.reset_index(drop= True)

In [25]:
df_sd[df_sd['Showroom'] == 'Semarang'].head(50)

,Tgl DO,Showroom,Observation,Trend,Monthly Seasonal,Yearly Seasonal
0,2017-01-04,Semarang,2.0,1.623461,-0.040852,-0.507455
1,2017-01-05,Semarang,1.0,1.627543,0.033714,-0.257455
2,2017-01-06,Semarang,2.0,1.631625,0.049823,-0.508825
3,2017-01-07,Semarang,0.0,1.635708,-0.142776,0.240490
4,2017-01-08,Semarang,1.0,1.639790,-0.127911,0.241175
5,2017-01-09,Semarang,1.0,1.643872,-0.023246,-0.006770
6,2017-01-10,Semarang,1.0,1.647955,0.052066,-0.256770
7,2017-01-11,Semarang,2.0,1.652037,0.066931,-0.508140
8,2017-01-12,Semarang,1.0,1.656119,-0.339678,-1.008140
9,2017-01-13,Semarang,1.0,1.660201,-0.204294,0.991860


In [27]:
df_sd.to_excel('dummy_seasonal_decompose.xlsx')